In [40]:
from reportlab.lib.pagesizes import letter
from reportlab.lib.units import inch
from reportlab.pdfgen import canvas
from reportlab.lib import colors
from PIL import Image
import barcode
from barcode.writer import ImageWriter
from io import BytesIO
from google.colab import files
import ipywidgets as widgets
from IPython.display import display

# Function to draw ID card
def generate_id_card(name, father_name, roll_number, batch_id, quarter, center, campus, city, timing, photo_path, background_path):
    # Increase ID Card size
    id_card_width = 5.0 * inch  # Increased width
    id_card_height = 3.0 * inch  # Increased height

    # Create PDF
    c = canvas.Canvas("id_card.pdf", pagesize=(id_card_width, id_card_height))

    # Set background image if provided
    if background_path:
        c.drawImage(background_path, 0, 0, width=id_card_width, height=id_card_height)

    # Draw a thick Border around the ID card (complete border)
    c.setStrokeColor(colors.black)
    c.setLineWidth(2)  # Adjust the thickness of the border
    c.rect(0.1 * inch, 0.1 * inch, id_card_width - 0.2 * inch, id_card_height - 0.2 * inch)  # Complete border with a small margin from edges

    # Set left side text fields with more spacing
    text_margin_x = 0.3 * inch  # More margin from the left
    text_margin_y = id_card_height - 0.3 * inch
    line_spacing = 0.25 * inch  # More line spacing

    c.setFont("Helvetica-Bold", 8)
    c.drawString(text_margin_x, text_margin_y, f"Name: {name}")
    c.drawString(text_margin_x, text_margin_y - line_spacing, f"Father's Name: {father_name}")
    c.drawString(text_margin_x, text_margin_y - 2 * line_spacing, f"Roll Number: {roll_number}")
    c.drawString(text_margin_x, text_margin_y - 3 * line_spacing, f"Batch ID: {batch_id}")
    c.drawString(text_margin_x, text_margin_y - 4 * line_spacing, f"Quarter: {quarter}")
    c.drawString(text_margin_x, text_margin_y - 5 * line_spacing, f"Center: {center}")
    c.drawString(text_margin_x, text_margin_y - 6 * line_spacing, f"Campus: {campus}")
    c.drawString(text_margin_x, text_margin_y - 7 * line_spacing, f"City: {city}")
    c.drawString(text_margin_x, text_margin_y - 8 * line_spacing, f"Timing: {timing}")

    # Insert photo on the right side with more spacing
    if photo_path:
        c.drawImage(photo_path, id_card_width - 1.7 * inch, id_card_height - 2.0 * inch, width=1.5 * inch, height=1.8 * inch)  # Adjusted size and spacing

    # Generate barcode and place below the photo with a good gap
    barcode_value = roll_number  # Roll Number as barcode data
    EAN = barcode.get_barcode_class('code128')
    ean = EAN(barcode_value, writer=ImageWriter())
    barcode_buffer = BytesIO()
    ean.write(barcode_buffer)

    barcode_img = Image.open(barcode_buffer)
    barcode_path = "barcode_image.png"
    barcode_img.save(barcode_path)

    # Place barcode below the photo with a clear gap
    c.drawImage(barcode_path, id_card_width - 1.7 * inch, 0.8 * inch, width=1.5 * inch, height=0.5 * inch)  # Proper gap between photo and barcode

    # Add "Authorized Signature" text below barcode on the right side, moved lower
    c.setFont("Helvetica-Bold", 8)
    c.drawString(id_card_width - 1.7 * inch, 0.2 * inch, "Authorized Signature")  # Moved lower

    # Add a hyperlink for the barcode
    c.linkURL("http://www.example.com", (id_card_width - 1.7 * inch, 0.8 * inch, id_card_width - 0.3 * inch, 1.3 * inch))  # Clickable barcode area

    # Save PDF
    c.showPage()
    c.save()
    print("ID Card PDF generated successfully!")

# Function to handle form submission and generate ID card
def on_submit(b):
    name = name_input.value
    father_name = father_name_input.value
    roll_number = roll_number_input.value
    batch_id = batch_id_input.value
    quarter = quarter_input.value
    center = center_input.value
    campus = campus_input.value
    city = city_input.value
    timing = timing_input.value

    # Extract uploaded file paths
    # Extract photo and background files from FileUpload widget
    if photo_upload.value:
        photo_filename = list(photo_upload.value.keys())[0]
        photo_path = photo_filename
        with open(photo_path, 'wb') as f:
            f.write(photo_upload.value[photo_filename]['content'])
    else:
        photo_path = None

    if background_upload.value:
        background_filename = list(background_upload.value.keys())[0]
        background_path = background_filename
        with open(background_path, 'wb') as f:
            f.write(background_upload.value[background_filename]['content'])
    else:
        background_path = None

    # Generate the ID card
    generate_id_card(name, father_name, roll_number, batch_id, quarter, center, campus, city, timing, photo_path, background_path)

    # Download the generated PDF
    files.download("id_card.pdf")

# Create input fields using ipywidgets
name_input = widgets.Text(description="Name:")
father_name_input = widgets.Text(description="Father's Name:")
roll_number_input = widgets.Text(description="Roll Number:")
batch_id_input = widgets.Text(description="Batch ID:")
quarter_input = widgets.Text(description="Quarter:")  # New field for Quarter
center_input = widgets.Text(description="Center:")
campus_input = widgets.Text(description="Campus:")
city_input = widgets.Text(description="City:")
timing_input = widgets.Text(description="Timing:")

# Create upload buttons for files (Photo, Background)
photo_upload = widgets.FileUpload(description="Upload Photo", accept="image/*")
background_upload = widgets.FileUpload(description="Upload Background", accept="image/*")

# Create button for submission
submit_button = widgets.Button(description="Generate ID Card")
submit_button.on_click(on_submit)

# Display the input form and upload buttons
display(name_input, father_name_input, roll_number_input, batch_id_input, quarter_input, center_input, campus_input, city_input, timing_input, photo_upload, background_upload, submit_button)


Text(value='', description='Name:')

Text(value='', description="Father's Name:")

Text(value='', description='Roll Number:')

Text(value='', description='Batch ID:')

Text(value='', description='Quarter:')

...

FileUpload(value={}, accept='image/*', description='Upload Background')

Button(description='Generate ID Card', style=ButtonStyle())

ID Card PDF generated successfully!


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install qrcode[pil]
!pip install pdfkit
!apt-get install wkhtmltopdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 2.4 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  avahi-daemon bind9-host bind9-libs geoclue-2.0 glib-networking glib-networking-common
  glib-networking-services gsettings-desktop-schemas iio-sensor-proxy libavahi-core7 libavahi-glib1
  libdaemon0 libevdev2 libfontenc1 libgudev-1.0-0 libhyphen0 libinput-bin libinput10
  libjson-glib-1.0-0 libjson-glib-1.0-common liblmdb0 libmaxminddb0 libmbim-glib4 libmbim-proxy
  libmd4c0 libmm-glib0 libmtdev1 libnl-genl-3-200 libnotify4 libnss-mdns libproxy1v5 libqmi-glib5
  libqmi-proxy libqt5core5a libqt5dbus5 libqt5gui5 libqt5network5 libqt5positioning5
  libqt5printsupport5 libqt5qml5 libqt5qmlmodels5 libqt5quick5 libqt5sensors5 libqt5svg5
  libqt5webchannel5 libqt5webkit5 libqt5widgets5 libs